In [1]:
pixnetFolder = {
    'pixnet_dataset/Google Search 關鍵字log/': ['PIXNET_search_log_000000000000.gz', 'PIXNET_search_log_000000000001.gz',
        'PIXNET_search_log_000000000002.gz', 'PIXNET_search_log_000000000003.gz', 'PIXNET_search_log_url_articles.gz'],
    'pixnet_dataset/HotelsCombined API/':  ['Hackathon_Hotels_TW_CN.csv', 'Hackathon_Hotels_TW_EN.csv', 
        'HotelsCombined API -Hackathon.pdf', 'Themes.xlsx'],
    'pixnet_dataset/HotelsCombined 導流 log - PIXNET 瀏覽 logs/': ['HotelsCombined_log.gz', 'HotelsCombined_log_referrer_url_articles.zip'],
    'pixnet_dataset/HotelsCombined 導流 log - 交易紀錄數量資料/': ['HotelsCombined_transaction_record.gz', 'HotelsCombined_Zone_欄位說明.pdf'],
    'pixnet_dataset/使用者瀏覽log/': ['PIXNET_visit_log_000000000000.gz', 'PIXNET_visit_log_000000000001.gz',
        'PIXNET_visit_log_000000000002.gz', 'PIXNET_visit_log_000000000003.gz', 'PIXNET_visit_log_000000000004.gz',
        'PIXNET_visit_log_000000000005.gz', 'PIXNET_visit_log_url_articles.gz'],
    'pixnet_dataset/使用者點擊log/': ['PIXNET_event_log_000000000000.gz', 'PIXNET_event_log_000000000001.gz',
        'PIXNET_event_log_000000000002.gz', 'PIXNET_event_log_000000000003.gz', 'PIXNET_event_log_000000000004.gz',
        'PIXNET_event_log_000000000005.gz', 'PIXNET_event_log_000000000006.gz', 'PIXNET_event_log_url_articles.zip'],
    'pixnet_dataset/文章資料集/': ['articles_182k.gz']
}

In [2]:
from data_extractor import *
import json
import re

In [3]:
with open('poi_data/scenic_spot_with_alias.json', encoding='utf-8-sig') as f:
    scenic_spot = json.load(f)
    scenic_spot = scenic_spot['XML_Head']['Infos']['Info']
    
with open('poi_data/poi_name.json', 'w', encoding='utf8') as f:
    scenic_spot = list([spot['Name'], *spot['alias']] for spot in scenic_spot)
    scenic_spot.sort()
    scenic_spot = scenic_spot[::-1]
    scenic_spot = [item for pair in scenic_spot for item in pair]
    scenic_spot = {item: index for index, item in enumerate(scenic_spot)}
    scenic_spot = list(scenic_spot.keys())

    json.dump(scenic_spot, f, indent=4, ensure_ascii=False)

In [4]:
import multiprocessing
dataset = PixnetArticle()
input_data = list(json.loads(data) for data in dataset if json.loads(data)['site_category'] == '國內旅遊')

In [5]:
def matching_poi(data):
    mapping = {}
    pattern = '|'.join(re.escape(loc) for loc in scenic_spot if len(loc) > 1)
    matched_generator = re.finditer(pattern, data['content'])
    for matched in matched_generator:
        if data['url'] not in mapping:
            mapping[data['url']] = [matched.group(0)]
        else:
            mapping[data['url']].append(matched.group(0))
    return mapping

result = []
with multiprocessing.Pool(max(int(multiprocessing.cpu_count()), 1)) as p:
    result.extend(p.map(matching_poi, input_data))
    p.close()
    
result = [data for data in result if data]

In [6]:
mapping = {}
for data in result:
    mapping = {**mapping, **data}
for url in mapping:
    mapping[url] = {item: index for index, item in enumerate(mapping[url]) if len(item) > 1}
    mapping[url] = list(mapping[url].keys())

In [7]:
with open('poi_data/scenic_spot_with_alias.json', encoding='utf-8-sig') as f:
    scenic_spot = json.load(f)
    scenic_spot = scenic_spot['XML_Head']['Infos']['Info']
    scenic_spot = {data['Name']: data['alias'] for data in scenic_spot}
    
for url in mapping:
    for index, loc in enumerate(mapping[url]):
        #Mapping name back to original
        for name in scenic_spot:
            if loc in scenic_spot[name]:
                mapping[url][index] = name

In [8]:
with open('poi_data/article2POIlist.json', 'w', encoding='utf8') as f:
    json.dump(mapping, f, indent=4, ensure_ascii=False)

In [9]:
count_mapping = {}
for data in mapping.values():
    data = set(data)
    count = 0
    for key in mapping:
        if set(mapping[key]) == data:
            count += 1
            
    count_mapping[str(data)] = [data, count]

In [10]:
count = 0
for keys in count_mapping:
    if len(count_mapping[keys][0]) > 1 and count_mapping[keys][1] > 1:
        count += 1
print(count)

1475


In [11]:
for index, data in enumerate(mapping):
    print(mapping[data])
    if index == 10:
        break

['武陵農場', '南澳農場', '露營區', '南化水庫', '皇后鎮森林']
['綠色隧道', '鶯歌車站']
['蓮花池']
['豐樂雕塑公園']
['鯤喜灣-親水公園']
['水頭碼頭', '模範街', '獅山砲陣地', '馬山觀測所', '沙美老街', '陽翟老街', '毋忘在莒', '莒光樓', '水頭聚落', '得月樓', '黃輝煌洋樓', '金水國小', '金門酒廠']
['國立自然科學博物館', '潭雅神綠園道']
['原住民文化會館']
['后里馬場', '后豐鐵馬道']
['三坑老街', '三坑自然生態公園']
['薰衣草森林']
